# Handy
## Test model

In [ ]:
import numpy as np


def calculate_angle_from_obj(a, b, c):
    a = (a.x, a.y)  # First
    b = (b.x, b.y)  # Mid - the angle corner
    c = (c.x, c.y)  # End

    return calculate_angle(a, b, c)


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid - the angle corner
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


In [26]:
import os
import random
import cv2
import numpy as np
from os import path
import mediapipe as mp
import pickle

import pandas as pd


mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

model_path = "handy_classifier.pkl"


if not path.exists("frames/"):
    print("Frames directory not found!")
    exit(-1)

with mp_holistic.Holistic(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as holistic, open(model_path, "rb") as f:
        model = pickle.load(f)
        items = os.listdir("frames/")
        items = [item for item in items if item.endswith(".png")]
        item = random.choice(items)

        print(f"File: {item}")

        frame = cv2.imread(path.join("frames", item))

         # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        if results.pose_landmarks is None:
            print("Pose not detected")
            exit(-1)
        
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=1, circle_radius=1),
        )
        # Calculate the angles
        # See train/angles.png for more info

        landmarks = results.pose_landmarks.landmark
        angles = []
        # Angle 0
        angles.append(
            calculate_angle_from_obj(landmarks[12], landmarks[14], landmarks[16])
        )
        # Angle 1
        angles.append(
            calculate_angle_from_obj(landmarks[11], landmarks[13], landmarks[15])
        )
        # Angle 2
        angles.append(
            calculate_angle_from_obj(landmarks[14], landmarks[12], landmarks[24])
        )
        # Angle 3
        angles.append(
            calculate_angle_from_obj(landmarks[13], landmarks[11], landmarks[23])
        )

        print(f"Angles: {', '.join([f'{angle:.2f}' for angle in angles])}")

        # Try to make pose guess
        X = pd.DataFrame([angles], columns=[f"angle_{angle_index}" for angle_index in range(len(angles))])
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]
        print(body_language_class)
        print(body_language_prob)

        cv2.imshow("Handy", image)
        while True:
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
cv2.destroyAllWindows()

File: 0_45.png
Angles: 178.02, 179.18, 10.66, 5.05
6
[0.1 0.  0.  0.  0.  0.  0.7 0.2 0.  0. ]
